In [9]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0-alpha0
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.16.2
pandas 0.25.0
sklearn 0.21.3
tensorflow 2.0.0-alpha0
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [10]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)


.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [11]:
from sklearn.model_selection import train_test_split

x_train_all, x_test, y_train_all, y_test = train_test_split(
    housing.data, housing.target, random_state = 7)
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_all, y_train_all, random_state = 11)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)


In [13]:
metric = keras.metrics.MeanSquaredError()
print(metric([5.], [2.]))
print(metric([0.], [1.]))
print(metric.result())

metric.reset_states()
print(metric([1.], [3.]))

tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [19]:

epochs = 100
batch_size = 32
steps_per_epoch = len(x_train_scaled) // batch_size
optimizer = keras.optimizers.SGD()
metric = keras.metrics.MeanSquaredError()

def random_batch(x, y, batch_size = 32):
    idx = np.random.randint(0, len(x), size=batch_size)
    return x[idx], y[idx]

model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu',
                      input_shape=x_train.shape[1:]),
    keras.layers.Dense(1),
])

for epoch in range(epochs):
    metric.reset_states()
    for step in range(steps_per_epoch):
        x_batch, y_batch = random_batch(x_train_scaled, y_train, batch_size)
        with tf.GradientTape() as tape:
            y_pred = model(x_batch)
            loss = tf.reduce_mean(keras.losses.mean_squared_error(y_batch, y_pred))
            metric(y_batch, y_pred)
        grads = tape.gradient(loss, model.variables)
        grads_and_vars = zip(grads, model.variables)
        optimizer.apply_gradients(grads_and_vars)
        print("\rEpoch", epoch, " train mse:", metric.result().numpy, end="")
    y_valid_pred = model(x_valid_scaled)
    valid_loss = tf.reduce_mean(
        keras.losses.mean_squared_error(y_valid_pred, y_valid))
    print("\t", "valid mse: ", valid_loss.numpy())
    

Epoch 0  train mse: <bound method _EagerTensorBase.numpy of <tf.Tensor: id=11913249, shape=(), dtype=float32, numpy=2.5924823>>	 valid mse:  1.6044737820297674gerTensorBase.numpy of <tf.Tensor: id=11894609, shape=(), dtype=float32, numpy=4.2373967>> 0  train mse: <bound method _EagerTensorBase.numpy of <tf.Tensor: id=11910129, shape=(), dtype=float32, numpy=2.7256985>>
Epoch 1  train mse: <bound method _EagerTensorBase.numpy of <tf.Tensor: id=11942230, shape=(), dtype=float32, numpy=1.3599908>>	 valid mse:  1.5116729006406346gerTensorBase.numpy of <tf.Tensor: id=11913590, shape=(), dtype=float32, numpy=1.2708968>>  train mse: <bound method _EagerTensorBase.numpy of <tf.Tensor: id=11918630, shape=(), dtype=float32, numpy=1.6617324>>1  train mse: <bound method _EagerTensorBase.numpy of <tf.Tensor: id=11923750, shape=(), dtype=float32, numpy=1.4976012>> 1  train mse: <bound method _EagerTensorBase.numpy of <tf.Tensor: id=11939750, shape=(), dtype=float32, numpy=1.3651767>>
Epoch 2  train 

Epoch

Epoch

Epoch 21  train mse: <bound method _EagerTensorBase.numpy of <tf.Tensor: id=12521850, shape=(), dtype=float32, numpy=1.2410139>>	 valid mse:  1.4101289682293918agerTensorBase.numpy of <tf.Tensor: id=12514090, shape=(), dtype=float32, numpy=1.2377132>>
Epoch 22  train mse: <bound method _EagerTensorBase.numpy of <tf.Tensor: id=12550831, shape=(), dtype=float32, numpy=1.297385>>	 valid mse:  1.4101464540657063erTensorBase.numpy of <tf.Tensor: id=12533311, shape=(), dtype=float32, numpy=1.2771407>> 22  train mse: <bound method _EagerTensorBase.numpy of <tf.Tensor: id=12543871, shape=(), dtype=float32, numpy=1.2843746>> 22  train mse: <bound method _EagerTensorBase.numpy of <tf.Tensor: id=12549231, shape=(), dtype=float32, numpy=1.2988964>>
Epoch 23  train mse: <bound method _EagerTensorBase.numpy of <tf.Tensor: id=12579812, shape=(), dtype=float32, numpy=1.2800565>>	 valid mse:  1.4090567875509337agerTensorBase.numpy of <tf.Tensor: id=12552612, shape=(), dtype=float32, numpy=1.2594131>> 2

Epoch 40  train mse: <bound method _EagerTensorBase.numpy of <tf.Tensor: id=13060009, shape=(), dtype=float32, numpy=1.2626067>>

...

Epoch

Epoch 83  train mse: <bound method _EagerTensorBase.numpy of <tf.Tensor: id=14303392, shape=(), dtype=float32, numpy=1.3092587>>

Epoch 83  train mse: <bound method _EagerTensorBase.numpy of <tf.Tensor: id=14308912, shape=(), dtype=float32, numpy=1.2798492>>

Epoch

Epoch 83  train mse: <bound method _EagerTensorBase.numpy of <tf.Tensor: id=14318672, shape=(), dtype=float32, numpy=1.2787623>>	 valid mse:  1.3884211784628364
Epoch 84  train mse: <bound method _EagerTensorBase.numpy of <tf.Tensor: id=14347653, shape=(), dtype=float32, numpy=1.2633066>>	 valid mse:  1.3886012794147173agerTensorBase.numpy of <tf.Tensor: id=14334853, shape=(), dtype=float32, numpy=1.2628586>>
Epoch 85  train mse: <bound method _EagerTensorBase.numpy of <tf.Tensor: id=14376634, shape=(), dtype=float32, numpy=1.2925999>>	 valid mse:  1.3875774672895886
Epoch 86  train mse: <bound method _EagerTensorBase.numpy of <tf.Tensor: id=14405615, shape=(), dtype=float32, numpy=1.2103269>>	 valid mse:  1.3894230563772556umpy of <tf.Tensor: id=14398335, shape=(), dtype=float32, numpy=1.2084848>>
Epoch 87  train mse: <bound method _EagerTensorBase.numpy of <tf.Tensor: id=14434596, shape=(), dtype=float32, numpy=1.2973431>>	 valid mse:  1.3883076131339809agerTensorBase.numpy of <tf.Te